In [1]:
import pandas as pd
from json_ntv import Ntv
from datetime import date, time, datetime

In [57]:
srs = [pd.Series([1,2,3], dtype='int64'),
       pd.Series([True, False, True], dtype='bool'),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[3,4], [5,6]]),  
       pd.Series([[3,4], {'a': 3, 'e':5}]),  
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns, UTC]'),
       pd.Series([1,2,3], dtype='int32'),
       pd.Series([1,2,3], dtype='UInt64'),
       pd.Series([1,2,3], dtype='float64'),
       pd.Series([1,2,3], dtype='object'),
       pd.Series(['az', 'er', 'cd'], dtype='string'),
       pd.Series([None, None, None])
]
for sr in srs:
    print(sr.equals(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series')))


True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False


In [53]:
sr = pd.Series([True, False, True])

print(sr.to_json(orient='records', date_format='iso', default_handler=str))
print(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series'))
print(pd.read_json(sr.to_json(orient='records', default_handler=str), typ='series'))


[true,false,true]
0     True
1    False
2     True
dtype: bool
0     True
1    False
2     True
dtype: bool


## mapping des types
Type json implicite (pas de ::name):
- json(number) - float64, int64
- json(string) - object(string)
- json(object) - object(dict)
- json(array) - object(tuple)
- json(bool) - bool
- json(null) - NaT

Type pandas spécifique (::name non maintenu sur pandas):
- datetime - datetime64[ns]
- intxx, uintxx - Intxx, UIntxx
- floatxx - floatxx
- string - string

Type pandas explicite (::name maintenu sur pandas):
- float64 - float64
- int64 - Int64
- object - object
- array - object
- bool - bool
- null - NaT

Type  externe (::name maintenu sur pandas)

A voir
- datetime64[ns, <tz>]
- category
- period[<freq>]
- Sparse
- Interval

## conversion 
Type json:
- -> read_json(sans dtype)
- <- to_json

Type pandas spécifique ou explicite:
- -> read_json(avec dtype)
- <- to_json

Type externe:
- -> to_obj(format='obj') -> Series
- <- from_obj() <- Series

In [58]:
a = pd.read_json('[1,2]', typ='series', dtype='Int32' )
print(type(a[0]))
print(a[0])

<class 'numpy.int32'>
1
